In [ ]:

# "Adam, Alice, Aria, Bill, George, Lilly, Reacheal, Sarah",  "Alex", "Arabella", "Bill L. Oxley", "BRIAN.wav", "Brittney", "Dramatic British.wav",
# "Heather Rey", "John Fernandes", "Laura", "Mark", "Matilda", "Roger", "Sally Ford", "SARAH", "Tim Rooney", "Todd Thomas", "Tyrone", "Realistic_Male", "realistic_male_2"
# "F5TTS_Male"

# ['en', 'es', 'fr', 'de', 'it', 'pt', 'pl', 'tr', 'ru', 'nl', 'cs', 'ar', 'zh-cn', 'hu', 'ko', 'ja', 'hi']

VOICE_TYPE = "SARAH"
LANGUAGE = "en"

SHEET_ID  = "1BGuyZ-mkJ0L2O_nztQko8OrYjs0MJBu98t2-57HdKjc"
SHEET_URL = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=csv'


In [ ]:
!pip install --upgrade tensorflow
# !pip install --upgrade numpy==1.22.0
!pip install gspread
!pip install cython>=0.29.30
!pip install torch>=2.1
!pip install anyascii>=0.3.0
!pip install pyyaml>=6.0
!pip install fsspec>=2023.6.0
!pip install aiohttp>=3.8.1
!pip install packaging>=23.1
!pip install TTS
!pip install gradio
!pip install openpyxl
!pip install gdown

In [ ]:
import torch
from TTS.api import TTS
import random
import os
import pandas as pd
import gdown
import random
import time
import shutil

In [ ]:
WORKIND_DIR = "Mouce_TTS_Outout"
SPEAKERS_DIR = os.path.join(f"./{WORKIND_DIR}", "Speakers")
WAV_FILES = str(os.path.join(f"./{WORKIND_DIR}", "Audios"))

if not os.path.exists(WORKIND_DIR): os.mkdir(WORKIND_DIR)
if not os.path.exists(SPEAKERS_DIR): os.mkdir(SPEAKERS_DIR)
if not os.path.exists(WAV_FILES): os.mkdir(WAV_FILES)


##############################################################################

# **MAIN CODE BELOW**


In [ ]:

class TEXT_TO_SPEECH:
    def __init__(self) -> None:
      device = "cuda" if torch.cuda.is_available() else "cpu"
      print(TTS().list_models())
      from unittest.mock import patch
      with patch('builtins.input', return_value='y'):
        self.tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)
      # self.tts = TTS("tts_models/multilingual/multi-dataset/xtts_v1.1").to(device)
      # self.tts = TTS("tts_models/bg/cv/vits").to(device)

      self.VOICES = {
        'Adam': '1dJA6oL9g8OPlFtWPWebos8EJssPhcF_U',
        'Alice': '1uSorJ01N5veelDiXrOJylMoH2Iu4MjFz',
        'Aria': '1XaNapPMehklx2WqYI2oHgKtEx6NZHeD0',
        'Bill': '1oMrRsqouYEB9TjArajC_yEUVtqniPTNv',
        'George': '1OCBJR5r1OLMPLGCY1GyaQ_7zBjDUBRX8',
        'Lily': '1j2es6mPX552NwblDK0TvuP99kU8IsWdA',
        'Reachel': '1SIfGSOO1M59emMQ1xtyXUuaD9bsGNWs3',
        'Sarah': '17NAJuGqQ6xcJJNxZgxoARWzQAprNDhhx',
        "Alex": "1mzlMLVS6K_YN3dZ-pKBysX7urx7UcEtQ",
        "SARAH": "1_o2dXyX3rJlJI_9R6Vkk1XisJnvYjKZ1",
        "Realistic_Male": "1GGnXLSmR2Y2MQF-wuodie4FbAxw876mH",
        "realistic_male_2": "1bdTSj6__QMCD-5rPWh9eFfqwzhTkqc6H",
        "F5TTS_Male": "1Mngf-Q63uP1X469iCKlsELWugSE8oS8B",
        "Realistic_Man_Voice_[Short]": "13OA70vrlziGfcUPC4fSmRw9i5uTZIQgl"
      }
      self.LANGUAGES = ['en', 'es', 'fr', 'de', 'it', 'pt', 'pl', 'tr', 'ru', 'nl', 'cs', 'ar', 'zh-cn', 'hu', 'ko', 'ja', 'hi']


    def transcribe(self, message, voice, lang):
      if lang not in self.LANGUAGES:
        print("LANGUAGE NOT FOUND... PLEASE SELECT VALID ONE")
        return None

      file_path = str(os.path.join(WAV_FILES, f"output_voice_{random.randrange(11111, 99999)}.wav"))
      self.tts.tts_to_file(text=message, speaker_wav=str(voice), language=lang, file_path=file_path)
      return file_path

def download_speaker(id, name):
  download_url = f'https://drive.google.com/uc?id={id}'
  output_file = f"{os.path.join(SPEAKERS_DIR, f'{name}.wav')}"
  gdown.download(download_url, output_file, quiet=False)
  time.sleep(2)


In [ ]:
AUDIO_FILES = []
FILTERED = []
TTS_OBJ = TEXT_TO_SPEECH()

url = SHEET_URL
df = pd.read_csv(url, header=None)
df_af = df.iloc[:, 0:6]
nested_list = df_af.values.tolist()
# print(nested_list)


In [ ]:
FILTERED_LiST = []
for i in nested_list:
  temp = [i[0], i[1], i[2]]
  FILTERED_LiST.append(temp)

# print(FILTERED_LiST)
nested_list = FILTERED_LiST



In [ ]:
for message in nested_list:
  temp = []
  for i in message:
    if not os.path.exists(f"{SPEAKERS_DIR}/{VOICE_TYPE}.wav"): download_speaker(TTS_OBJ.VOICES[VOICE_TYPE], VOICE_TYPE)
    file_path = TTS_OBJ.transcribe(i, f"{SPEAKERS_DIR}/{VOICE_TYPE}.wav", LANGUAGE)
    if file_path: temp.append(os.path.basename(file_path))
  AUDIO_FILES.append(temp)
# print(AUDIO_FILES)


In [ ]:
RESULTS = []
print(AUDIO_FILES)
for index, i in enumerate(nested_list):
  RESULTS.append(i + AUDIO_FILES[index])


In [ ]:
df = pd.DataFrame(RESULTS)
output_file =  os.path.join(WORKIND_DIR, 'data.xlsx')
df.to_excel(output_file, index=False, header=False)
print(f"Data has been saved to {output_file}")

In [ ]:
import shutil
from IPython.display import FileLink

folder_name = WORKIND_DIR
zip_name = 'text_to_speech.zip'
shutil.make_archive(zip_name.replace('.zip', ''), 'zip', folder_name)

display(FileLink(zip_name))


In [ ]:
# TTS_OBJ = TEXT_TO_SPEECH()
# download_speaker(TTS_OBJ.VOICES[VOICE_TYPE], VOICE_TYPE)

# temp = """
# I don't really care what you call me. I've been a silent spectator, watching species evolve, empires rise and fall. But always remember, I am mighty and enduring. Respect me and I'll nurture you; ignore me and you shall face the consequences.
# """

# file_path = TTS_OBJ.transcribe(temp, f"{SPEAKERS_DIR}/{VOICE_TYPE}.wav", LANGUAGE)
# file_path